<a href="https://colab.research.google.com/github/SinKasula/MachineLearning/blob/master/Kfold_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
import nn
from copy import deepcopy
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append('drive/My Drive/ML_Assign3/')
import nn

In [0]:

path = r"drive/My Drive/ML_Assign3/"
df = pd.read_csv(path+'/IN_Metro_Traffic.csv')
df = df.reindex(np.random.permutation(df.index))

In [0]:
len(df)

33750

In [0]:
def isHolidayBool(x):
    if x== 'None':
        return 0
    else:
        return 1
    
df.is_holiday= df.is_holiday.apply(lambda x : isHolidayBool(x))


weather_type_dic = {'Clouds':1, 'Clear':0, 'Mist':2, 'Rain':5, 'Snow':10, 'Drizzle':6, 'Haze':3,
                    'Fog':4, 'Thunderstorm':9, 'Smoke':7, 'Squall':8}

df.weather_type = df.weather_type.apply(lambda x : weather_type_dic[x])



def isWeekend(day):
    if day in [6,7]:
        return 1
    else:
        return 0
    
df["date_time"] = df["date_time"].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
df["year"] = df["date_time"].apply(lambda x: x.year)
df["month"] = df["date_time"].apply(lambda x: x.month)
df["day"] = df["date_time"].apply(lambda x: x.day)
df["hour"] = df["date_time"].apply(lambda x: x.hour)
df["weekday"] = df["date_time"].apply(lambda x: x.isoweekday())
df["is_weekend"] = df["weekday"].apply(lambda x: isWeekend(x))

df = df.drop(['date_time','weather_description'],axis = 1)

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33750 entries, 200 to 12011
Data columns (total 19 columns):
is_holiday             33750 non-null int64
air_pollution_index    33750 non-null int64
humidity               33750 non-null int64
wind_speed             33750 non-null int64
wind_direction         33750 non-null int64
visibility_in_miles    33750 non-null int64
dew_point              33750 non-null int64
temperature            33750 non-null float64
rain_p_h               33750 non-null float64
snow_p_h               33750 non-null float64
clouds_all             33750 non-null int64
weather_type           33750 non-null int64
traffic_volume         33750 non-null int64
year                   33750 non-null int64
month                  33750 non-null int64
day                    33750 non-null int64
hour                   33750 non-null int64
weekday                33750 non-null int64
is_weekend             33750 non-null int64
dtypes: float64(3), int64(16)
memory usage: 5.1

In [0]:
df.columns

Index(['is_holiday', 'air_pollution_index', 'humidity', 'wind_speed',
       'wind_direction', 'visibility_in_miles', 'dew_point', 'temperature',
       'rain_p_h', 'snow_p_h', 'clouds_all', 'weather_type', 'traffic_volume',
       'year', 'month', 'day', 'hour', 'weekday', 'is_weekend'],
      dtype='object')

In [0]:
X_col = ['is_holiday', 'air_pollution_index', 'humidity', 'wind_speed','wind_direction', 'visibility_in_miles', 'dew_point', 'temperature','rain_p_h', 'snow_p_h', 'clouds_all', 'weather_type', 'year', 'month', 'day', 'hour', 'weekday', 'is_weekend']
Y_col = ['traffic_volume']

In [0]:
hidden_layers = [10, 15, 20]
iterations = [1000, 5000, 10000]
alpha = [10, 1, 0.01]

param_list = [[i, j, k] for i in hidden_layers  
                 for j in iterations 
                 for k in alpha] 

len(param_list)

27

In [0]:
def data_partition(df, k):
    part = len(df)//k
    datasets =[]
    for i in range(k):

        if i == k-1:
            datasets.append(df[part*i:])
        else:
             datasets.append(df[part*i:part*(i+1)])
    return datasets

In [0]:
def K_CV(data, k, X, Y, in_layers, out_layers, param_list):
      ds = data_partition(data, k)
      test_errors = []
      best_params = []
      input_layers= in_layers
      output_layers = out_layers
      for i in range(k):
              testset = ds[i]
              X_test = testset[X].as_matrix()
              Y_test = testset[Y].as_matrix().reshape(-1,1)    
              
              rem = []
              for p in range(k):
                      if p !=i:
                          rem.append(ds[p])
          
              for j in range(len(rem)):
                      rem_copy = deepcopy(rem)
                      validate_set = rem[j]
                      trainset = rem_copy.pop(j)

                      X_train = trainset[X].as_matrix()
                      Y_train = trainset[Y].as_matrix().reshape(-1,1)
                      X_validate  =  validate_set[X].as_matrix()
                      Y_validate =  validate_set[Y].as_matrix().reshape(-1,1)

                      parms_errors = []
                      for l in range(len(param_list)):
                              params = param_list[l]
                              # Initializing a class
                              cls  = nn.NeuralNet([input_layers, params[0], output_layers], params[2])
                              cls.train(X_train, Y_train, niter= params[1])
                             
                              #Predicting a the output of testset
                              Pred = cls.use(X_validate)
                              err = np.mean(np.abs(Pred - Y_validate))
                              #print(err)
                              parms_errors.append(err)
                             
                        
                      
                      b_parm = param_list[parms_errors.index(min(parms_errors))]
                      print(parms_errors)
                      print(b_parm)
                      best_params.append(b_parm)
                      cls_best  = nn.NeuralNet([input_layers, b_parm[0] , output_layers], b_parm[2])
                      cls_best.train(X_train, Y_train, niter= b_parm[1])
                      Pred_test = cls.use(X_test)
                      err = np.mean(np.abs(Pred_test - Y_test))
                      test_errors.append(err)
                      print(err)
                      
      print("The best param set is ")
      zkl = best_params[test_errors.index(min(test_errors))]
      print(zkl)
      return zkl

In [0]:
# # SAMPLE DATA

X = np.linspace(-1000,1000, 500).reshape((-1, 1))
T = ((-0.75 * X) + 750)
data = np.hstack([X, T])
data = pd.DataFrame(data, columns=['X', 'T'])

In [0]:
sample_data_params = K_CV(data= data,k=5, X= ['X'], Y= ['T'], in_layers=1, out_layers=1, param_list=param_list)

[66.7290921227239, 0.27735261843450415, 0.06292660204466075, 14.054713543174735, 0.3167955049293789, 0.07951209233817849, 48.843553305602626, 0.11735752209783186, 0.148091555160305, 20.904431470460572, 0.1195542181023086, 0.058394763311284806, 7.394554444343711, 0.4207525451316883, 0.040084328590743325, 29.7209639275954, 0.2629454596422386, 0.04945900073974599, 3.012774088475395, 0.21520404051516492, 0.058626864566233505, 28.411481122427627, 0.018581289468600062, 0.09888804667425802, 7.318327498700168, 0.13497243734889822, 0.09167125704560021]
[20, 5000, 1]
48.37122835358932
[20.06036690221445, 0.19017662411378639, 0.31442423312547474, 30.446043893057258, 0.2436534747959638, 0.2703208376320606, 7.656820971434761, 0.33134821633288425, 0.1456856938078988, 41.33702455890376, 0.04444729287089331, 0.23124218295551713, 9.582915949533797, 0.3025725723424034, 0.15720690407528765, 48.126863288917946, 0.09736596164393518, 0.28637678136266914, 19.806028896702728, 0.04778706855486575, 0.2308821285

In [0]:
## Actual data hyper parameters
df_params = K_CV(data= df, k=5, X= X_col, Y= Y_col, in_layers= len(X_col), out_layers=1,
                 param_list = param_list)

[764.98, 512.91, 526.84, 1645.35, 431.49, 431.16, 1134.5, 369.48, 369.09, 1668.62, 493.12, 388.77, 1660.83, 336.69, 319.92, 909.69, 316.86, 336.44, 903.75, 423.05, 419.19, 1561.72, 307.18, 305.77, 1595.73, 322.31, 298.61, 1749.01, 478.15, 540.42, 1683.63, 385.76, 476.26, 1656.58, 348.39, 362.62, 1604.31, 504.58, 422.5, 1364.67, 353.96, 459.63, 1599.27, 309.32, 360.9, 1606.84, 401.69, 410.78, 1652.9, 308.41, 318.0, 1360.13, 300.72, 289.74, 1708.72, 480.05, 554.05, 1663.73, 472.56, 370.59, 1107.76, 370.37, 370.66, 1540.49, 485.46, 524.88, 1632.78, 329.28, 332.0, 1688.99, 320.13, 312.07, 1541.52, 408.98, 420.25, 1530.78, 342.36, 301.07, 1092.23, 299.52, 299.18, 1740.77, 486.3, 544.4, 1337.58, 389.51, 380.31, 951.86, 440.97, 349.51, 1574.01, 485.85, 437.63, 1603.23, 315.14, 327.41, 1595.09, 304.45, 336.95, 1568.24, 401.46, 372.99, 1569.33, 294.44, 288.56, 1576.55, 292.46, 285.45]
[20, 5000, 0.01]
363.79439575558996
[1607.47, 536.51, 505.65, 1683.67, 524.87, 423.05, 1693.02, 385.29, 395.31,

In [0]:
# clsf1  = nn.NeuralNet([18, 1, 1])

In [0]:
# _test = datasets[0]
# _train = df[~df.index.isin(datasets[0].index)]

In [0]:
# X_train = _train.drop('traffic_volume', axis=1)
# Y_train = _train['traffic_volume']

# X_test = _test.drop('traffic_volume', axis=1)
# Y_test = _test['traffic_volume']

# X_train_mat = X_train.as_matrix()
# X_test_mat = X_test.as_matrix()
# Y_train_mat = Y_train.as_matrix()
# Y_test_mat = Y_test.as_matrix()

# Y_train_mat = Y_train_mat.reshape(-1,1)
# print(Y_train_mat.shape)
# Y_test_mat = Y_test_mat.reshape(-1,1)
# print(Y_test_mat.shape)

In [0]:
# X_train.shape
# Y_train.shape

In [0]:
# clsf1.train(X_train_mat, Y_train_mat, _lambda = 0.001)

In [0]:
# Y_pred = clsf1.use(X_test_mat)

In [0]:
# #np.sum(np.square(Y_pred-Y_test_mat))/len(Y_pred)

# np.mean(np.abs(Y_test_mat - Y_pred)) 